In [29]:
import os
import posixpath
import xarray as xr
import rioxarray
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

In [7]:
data_root = 'D:/OneDrive/Documents/Cours/4A/SFE/data/KH-5_ARGON_images'
products = []

for x in os.listdir(data_root):
    if os.path.isdir(posixpath.join(data_root, x)):
        products.append(x)
        
products

['DS09034A007MC018', 'DS09058A024MC013']

In [14]:
image_path = posixpath.join(data_root, products[1], products[1] + "_a.tif")
image = rioxarray.open_rasterio(image_path)
image

<xarray.DataArray (band: 1, y: 19912, x: 19332)> Size: 385MB
[384938784 values with dtype=uint8]
Coordinates:
  * band         (band) int32 4B 1
  * x            (x) float64 155kB 0.5 1.5 2.5 ... 1.933e+04 1.933e+04 1.933e+04
  * y            (y) float64 159kB 0.5 1.5 2.5 ... 1.991e+04 1.991e+04 1.991e+04
    spatial_ref  int32 4B 0
Attributes:
    TIFFTAG_SOFTWARE:        Adobe Photoshop CC 2018 (Windows)
    TIFFTAG_DATETIME:        2019:12:03 22:13:38
    TIFFTAG_XRESOLUTION:     1432.3507
    TIFFTAG_YRESOLUTION:     1432.3507
    TIFFTAG_RESOLUTIONUNIT:  3 (pixels/cm)
    scale_factor:            1.0
    add_offset:              0.0

# Internal Orientation

In [71]:
def image_to_fiducial_coordinates(x, y, xc, yc, alpha, delta_xi, delta_eta):
    rotation_matrix = np.array([
        [delta_xi * np.cos(alpha), delta_eta * np.sin(alpha)],
        [-delta_xi * np.sin(alpha), delta_eta * np.cos(alpha)]
    ])
    
    xi_eta = np.zeros((2, 1))
    xi_eta = rotation_matrix @ np.array([[x - xc], [y - yc]])
    return xi_eta[0, 0], xi_eta[1, 0]


def fiducial_to_image_coordinates(xi, eta, xc, yc, alpha, delta_xi, delta_eta):
    rotation_matrix = np.array([
        [delta_xi * np.cos(alpha), delta_eta * np.sin(alpha)],
        [-delta_xi * np.sin(alpha), delta_eta * np.cos(alpha)]
    ])
    
    x_y = np.zeros((2, 1))
    x_y = np.linalg.inv(rotation_matrix) @ np.array([[xi + xc], [eta + yc]])
    return x_y[0, 0], x_y[1, 0]

x, y = 1, 0
xc_t, yc_t = 0, 0
alpha_t = 30 * np.pi/180
delta_xi_t, delta_eta_t = 1, 1
xi, eta = image_to_fiducial_coordinates(x, y, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
x1, y1 = fiducial_to_image_coordinates(xi, eta, xc_t, yc_t, alpha_t, delta_xi_t, delta_eta_t)
x1, y1, xi, eta

(1.0, -5.551115123125783e-17, 0.8660254037844387, -0.49999999999999994)

In [ ]:
def objective_function(params, FMs_fiducial_true_coords, FMs_image_true_coords):
    assert FMs_fiducial_true_coords.shape == FMs_image_true_coords.shape, "Fiducial and image coordinates must have the same shape."
    assert FMs_fiducial_true_coords.shape[1] == 2, "Fiducial and image coordinates must have two columns."
    
    p = FMs_fiducial_true_coords.shape[0]
    xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
    
    FMs_fiducial_inferred_coords = np.zeros((p, 2))
    for i in range(p):
        x_true, y_true = FMs_image_true_coords[i, 0], FMs_image_true_coords[i, 1]
        
        xi, eta = image_to_fiducial_coordinates(x_true, y_true, xc, yc, alpha, delta_xi, delta_eta)
        FMs_fiducial_inferred_coords[i, :] = np.array([xi, eta])
        
    res = 1/2 * np.linalg.norm(FMs_fiducial_inferred_coords - FMs_fiducial_true_coords, axis=1) ** 2
    res = np.sum(res)
    return res

FMs_fiducial_coords = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
]) * 12

FMs_image_coords = np.array([
    [0, 0],
    [10.392304845413264, -5.999999999999999],
    [5.999999999999999, 10.392304845413264],
    [16.392304845413264, 4.392304845413265]
])

res = opt.least_squares(
    objective_function,
    x0=[0, 0, 0, 1, 1],
    args=(FMs_fiducial_coords, FMs_image_coords),
)
params = res.x # problem w/ alpha → remove it from the optimization process?

xc, yc, alpha, delta_xi, delta_eta = params[0], params[1], params[2], params[3], params[4]
x, y = 1, 0
xi_t, eta_t = 0.8660254037844387, -0.49999999999999994
xi, eta = image_to_fiducial_coordinates(x, y, xc, yc, alpha, delta_xi, delta_eta)
xi, xi_t, eta, eta_t

(0.8643653759733506,
 0.8660254037844387,
 0.5008300115234054,
 -0.49999999999999994)

In [78]:
alpha_t

0.5235987755982988

In [58]:
params

array([ 0.0010946 , -0.00164939, -0.52349819,  1.00007189,  0.99995241])

In [69]:
a = np.array([
    [1, 2],
    [3, 4],
    [5, 6]
])
np.linalg.norm(a, axis=1).sum()

15.046317653406444